Etapa 1: Carregando os modelos e bibliotecas

In [10]:
import pandas as pd
import numpy as np
from IPython.display import display
import joblib
import pickle

Etapa 2: Geração dos arquivos para simulação como modelo salvo

In [7]:
# Gerar dados simulados
np.random.seed(42)
n_samples = 5

data_teste = pd.DataFrame({
    'gender': np.random.choice(['Male', 'Female'], n_samples),
    'SeniorCitizen': np.random.choice([0, 1], n_samples),
    'Partner': np.random.choice(['Yes', 'No'], n_samples),
    'Dependents': np.random.choice(['Yes', 'No'], n_samples),
    'tenure': np.random.randint(0, 72, n_samples),
    'PhoneService': np.random.choice(['Yes', 'No'], n_samples),
    'MultipleLines': np.random.choice(['Yes', 'No', 'No phone service'], n_samples),
    'InternetService': np.random.choice(['DSL', 'Fiber optic', 'No'], n_samples),
    'OnlineSecurity': np.random.choice(['Yes', 'No', 'No internet service'], n_samples),
    'OnlineBackup': np.random.choice(['Yes', 'No', 'No internet service'], n_samples),
    'DeviceProtection': np.random.choice(['Yes', 'No', 'No internet service'], n_samples),
    'TechSupport': np.random.choice(['Yes', 'No', 'No internet service'], n_samples),
    'StreamingTV': np.random.choice(['Yes', 'No', 'No internet service'], n_samples),
    'StreamingMovies': np.random.choice(['Yes', 'No', 'No internet service'], n_samples),
    'Contract': np.random.choice(['Month-to-month', 'One year', 'Two year'], n_samples),
    'PaperlessBilling': np.random.choice(['Yes', 'No'], n_samples),
    'PaymentMethod': np.random.choice(['Electronic check', 'Mailed check', 'Bank transfer (automatic)', 'Credit card (automatic)'], n_samples),
    'MonthlyCharges': np.round(np.random.uniform(20, 120, n_samples), 2),
})

In [ ]:
# A coluna 'TotalCharges' deve ser calculada após o DataFrame ser criado,
# pois usa outras colunas como base.
data_teste['TotalCharges'] = np.round(data_teste['MonthlyCharges'] * data_teste['tenure'] + np.random.uniform(-20, 20, n_samples), 2)

In [9]:
# Visualizar o DataFrame usando a função 'display' do IPython
print("--- Dados Simulados para Previsão ---")
display(data_teste)

--- Dados Simulados para Previsão ---


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,Male,1,Yes,Yes,21,No,No,No,No,No internet service,No internet service,Yes,No,Yes,One year,No,Mailed check,109.48,2280.89
1,Female,0,Yes,No,52,No,No,No,No,Yes,Yes,No,Yes,No internet service,One year,No,Mailed check,79.79,4142.09
2,Male,0,Yes,No,1,No,Yes,No,No internet service,No internet service,Yes,No,No,No internet service,One year,No,Credit card (automatic),112.19,107.74
3,Male,0,Yes,No,29,Yes,Yes,Fiber optic,No,Yes,No internet service,No,No internet service,No,One year,Yes,Mailed check,28.85,827.50
4,Male,1,No,Yes,37,Yes,Yes,No,No internet service,No internet service,No,Yes,No internet service,Yes,One year,Yes,Mailed check,39.60,1478.35


In [11]:
# Cópia de segurança
df_simulado_proc = data_teste.copy()

# 1. Remover colunas desnecessárias
colunas_remover = ['customerID', 'Churn'] if 'customerID' in df_simulado_proc.columns else ['Churn']
df_simulado_proc = df_simulado_proc.drop(columns=colunas_remover, errors='ignore')

# 2. Corrigir TotalCharges (caso exista)
if 'TotalCharges' in df_simulado_proc.columns:
    df_simulado_proc['TotalCharges'] = pd.to_numeric(df_simulado_proc['TotalCharges'], errors='coerce')
    df_simulado_proc['TotalCharges'] = df_simulado_proc['TotalCharges'].fillna(df_simulado_proc['TotalCharges'].median())

# 3. Variáveis binárias como 0 e 1
variaveis_binarias = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
for col in variaveis_binarias:
    if col in df_simulado_proc.columns:
        df_simulado_proc[col] = df_simulado_proc[col].map({'Yes': 1, 'No': 0})

# 4. Variáveis com "No internet service" ou "No phone service"
substituir_no_service = [
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines'
]
for col in substituir_no_service:
    if col in df_simulado_proc.columns:
        df_simulado_proc[col] = df_simulado_proc[col].replace({'No internet service': 'No', 'No phone service': 'No'})

# 5. Codificar variáveis categóricas
colunas_categoricas = df_simulado_proc.select_dtypes(include='object').columns.tolist()
df_simulado_proc = pd.get_dummies(df_simulado_proc, columns=colunas_categoricas, drop_first=True)

# 6. Garantir que não há valores nulos
df_simulado_proc = df_simulado_proc.fillna(0)

# 7. Verificar formato final
print(f"Shape final do dataset processado: {df_simulado_proc.shape}")
df_simulado_proc.head()


Shape final do dataset processado: (5, 17)


,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,gender_Male,MultipleLines_Yes,InternetService_No,OnlineBackup_Yes,DeviceProtection_Yes,TechSupport_Yes,StreamingTV_Yes,StreamingMovies_Yes,PaymentMethod_Mailed check
0,1,1,1,21,0,0,109.48,2280.89,True,False,True,False,False,True,False,True,True
1,0,1,0,52,0,0,79.79,4142.09,False,False,True,True,True,False,True,False,True
2,0,1,0,1,0,0,112.19,107.74,True,True,True,False,True,False,False,False,False
3,0,1,0,29,1,1,28.85,827.50,True,True,False,True,False,False,False,False,True
4,1,0,1,37,1,1,39.60,1478.35,True,True,True,False,False,True,False,True,True


In [12]:
# Converter valores booleanos para inteiros (True → 1, False → 0)
df_simulado_proc = df_simulado_proc.astype(int)

# Verificando resultado
df_simulado_proc.head()


,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,gender_Male,MultipleLines_Yes,InternetService_No,OnlineBackup_Yes,DeviceProtection_Yes,TechSupport_Yes,StreamingTV_Yes,StreamingMovies_Yes,PaymentMethod_Mailed check
0,1,1,1,21,0,0,109,2280,1,0,1,0,0,1,0,1,1
1,0,1,0,52,0,0,79,4142,0,0,1,1,1,0,1,0,1
2,0,1,0,1,0,0,112,107,1,1,1,0,1,0,0,0,0
3,0,1,0,29,1,1,28,827,1,1,0,1,0,0,0,0,1
4,1,0,1,37,1,1,39,1478,1,1,1,0,0,1,0,1,1


In [13]:
df_simulado_proc = df_simulado_proc.applymap(lambda x: int(x) if isinstance(x, bool) else x)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_18376\1718604939.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_simulado_proc = df_simulado_proc.applymap(lambda x: int(x) if isinstance(x, bool) else x)


Etapa 3: Carregar o modelo treinado

In [14]:
# Usando joblib
modelo_rf = joblib.load('/content/modelo_random_forest_joblib.pkl')

# Ou, se preferir usar pickle:
# with open('/content/modelo_random_forest_pickle.pkl', 'rb') as f:
#     modelo_rf = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/content/modelo_random_forest_joblib.pkl'

Etapa 4: Fazer previsões

In [ ]:
# Previsão binária
predicoes = modelo_rf.predict(dados_prontos)

# Probabilidade de churn (classe 1)
probabilidades = modelo_rf.predict_proba(dados_prontos)[:, 1]

# Juntar os resultados em um DataFrame
resultado = df_simulado.copy()
resultado['Churn_Previsto'] = predicoes
resultado['Probabilidade_Churn'] = probabilidades


Etapa 5: Exibir os resultados

In [ ]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
display(resultado[['Churn_Previsto', 'Probabilidade_Churn'] + list(df_simulado.columns)])